In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Pancreatic_Cancer"
cohort = "GSE124069"

# Input paths
in_trait_dir = "../../input/GEO/Pancreatic_Cancer"
in_cohort_dir = "../../input/GEO/Pancreatic_Cancer/GSE124069"

# Output paths
out_data_file = "../../output/preprocess/Pancreatic_Cancer/GSE124069.csv"
out_gene_data_file = "../../output/preprocess/Pancreatic_Cancer/gene_data/GSE124069.csv"
out_clinical_data_file = "../../output/preprocess/Pancreatic_Cancer/clinical_data/GSE124069.csv"
json_path = "../../output/preprocess/Pancreatic_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"BRD4 inhibitor JQ1 increase the sensitivity of arsenic trioxide in pancreatic cancer"
!Series_summary	"Pancreatic cancer is a deadliest type of malignancy, largely due to lack of effective intervention. We here report a pair of agents, ATO and JQ1, which synergistically induce apoptosis in the malignancy. Through global and molecular approaches, we have provided evidence that these agents are both able to modulate ER stress and autophagy in the cancer, probably acting in different ways. Cross-talks between ER stress and autophagy are implicated during ATO/ATO plus JQ1 induced apoptosis, in which NRF2 appears to play a central role.   of the globe transcriptional profiles of ATO regulated genes in breast, colon and lung cancer cells with different p53 status. We find p53 wild type cells are resistant to ATO induced globe dynamic transcriptional changes, thus resistant to ATO induced cell growth inhibition. P53 inhibitor PFTα releases p53 mediated t

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# This dataset appears to contain gene expression data as it mentions "Total RNA was collected and
# profiled to Affymetrix microarray" in the Series_overall_design
is_gene_available = True

# 2. Variable Availability and Data Type Conversion

# 2.1 For Trait (Pancreatic Cancer)
# All samples are pancreatic cancer cell lines according to the background info
# Trait should be binary, but since all samples are pancreatic cancer, this is a constant feature
trait_row = None  # No variation in disease state - all are pancreatic cancer

# 2.2 For Age
# No information on age - these are cell lines
age_row = None

# 2.3 For Gender
# No information on gender - these are cell lines
gender_row = None

# Conversion functions
def convert_trait(value):
    """
    Convert trait values.
    """
    if value is None:
        return None
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    if 'pancreatic cancer' in value.lower():
        return 1
    return None

def convert_age(value):
    """
    Convert age values.
    """
    # Not applicable for this dataset
    return None

def convert_gender(value):
    """
    Convert gender values.
    """
    # Not applicable for this dataset
    return None

# 3. Save Metadata
# Check if trait data is available (if trait_row is not None)
is_trait_available = trait_row is not None

# Validate and save cohort info
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Skip this step as trait_row is None (no clinical variation exists in this dataset)


False

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row IDs (gene or probe identifiers) for future observation.
print(gene_data.index[:20])


Index(['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at',
       '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at',
       '1494_f_at', '1552256_a_at', '1552257_a_at', '1552258_at', '1552261_at',
       '1552263_at', '1552264_a_at', '1552266_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Looking at the gene identifiers in the gene expression data
# These identifiers (like '1007_s_at', '1053_at', etc.) are not human gene symbols
# They are Affymetrix probe IDs, which need to be mapped to human gene symbols
# These appear to be Affymetrix HG-U133 array probe IDs

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database'], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /DEFINITION=HSP70B Human heat-shock protein HSP70B' gene", 'X69699 /FEATURE= /DEF

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify columns for probe IDs and gene symbols in the annotation dataframe
prob_col = 'ID'  # The probe identifiers are stored in the 'ID' column
gene_col = 'Gene Symbol'  # The gene symbols are stored in the 'Gene Symbol' column

# 2. Get a gene mapping dataframe by extracting these two columns
gene_mapping = get_gene_mapping(gene_annotation, prob_col, gene_col)

# 3. Convert probe-level measurements to gene expression data by applying the gene mapping
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Preview the gene-level expression data
print("Gene expression data preview (after mapping):")
print(gene_data.index[:20])  # Print the first 20 gene symbols
print(f"Total number of genes after mapping: {len(gene_data)}")


Gene expression data preview (after mapping):
Index(['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1', 'A2ML1', 'A2MP1',
       'A4GALT', 'A4GNT', 'AA06', 'AAAS', 'AACS', 'AACSP1', 'AADAC', 'AADACL2',
       'AADACP1', 'AADAT', 'AAED1', 'AAGAB', 'AAK1'],
      dtype='object', name='Gene')
Total number of genes after mapping: 21278


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# Since we determined in Step 2 that trait_row is None (no clinical variation)
# and is_trait_available is False, we can't proceed with linking clinical and gene data
# Instead, we need to create dummy clinical data with a constant trait value

# Create minimal clinical dataframe with samples matching gene data
sample_ids = normalized_gene_data.columns
dummy_clinical_df = pd.DataFrame(index=['Pancreatic_Cancer'], 
                                  columns=sample_ids,
                                  data=[[1] * len(sample_ids)])  # All samples are pancreatic cancer

print(f"Created dummy clinical data with shape: {dummy_clinical_df.shape}")

# 2. Link the clinical and genetic data (even though clinical data is just a constant)
linked_data = geo_link_clinical_genetic_data(dummy_clinical_df, normalized_gene_data)
print(f"Linked data shape: {linked_data.shape}")

# Identify trait column
trait_col = linked_data.columns[0]
print(f"Using trait column: {trait_col}")

# 3. Handle missing values in the linked data
linked_data = handle_missing_values(linked_data, trait_col)
print(f"Shape after handling missing values: {linked_data.shape}")

# 4. Determine whether the trait and demographic features are severely biased
# Since we know all samples are pancreatic cancer, this trait is severely biased
is_trait_biased = True
print("The trait is severely biased (all samples are pancreatic cancer).")
unbiased_linked_data = linked_data

# 5. Conduct quality check and save the cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=False,  # We previously determined trait data is not available (no variation)
    is_biased=is_trait_biased, 
    df=unbiased_linked_data,
    note="Dataset contains gene expression data for pancreatic cancer cell lines, but lacks normal controls."
)

# 6. Since the data is not usable (no trait variation), don't save it
if is_usable:
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    # Save the data
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Data quality check failed. Dataset lacks necessary trait variation for association studies.")

Normalized gene data saved to ../../output/preprocess/Pancreatic_Cancer/gene_data/GSE124069.csv
Created dummy clinical data with shape: (1, 30)
Linked data shape: (30, 19846)
Using trait column: Pancreatic_Cancer


Shape after handling missing values: (30, 19846)
The trait is severely biased (all samples are pancreatic cancer).
Data quality check failed. Dataset lacks necessary trait variation for association studies.
